# Hypothesis

This notebook serves as a demo of the basic features and functionality of the Python tool **Hypothesis**.

## What is Hypothesis

Hypothesis is a tool that allows us to --- **TODO**

## 1. Basic Usage: `@given` and Strategies

The core of Hypothesis is the `@given` decorator, which takes **strategies** that define how to generate data. Let's test a simple `add` function.

In [ ]:
import hypothesis
from hypothesis import given, strategies as st

def add(a, b):
    """A simple function to add two numbers."""
    return a + b

# This is a property-based test!
# It states that for any two integers a and b, add(a, b) should equal a + b.
@given(a=st.integers(), b=st.integers())
def test_addition_property(a, b):
    print(f"Testing with a={a}, b={b}") # See what Hypothesis tries!
    assert add(a, b) == a + b

# To run this in a notebook, we can call the test function directly.
# In a real project, you'd use a test runner like pytest.
test_addition_property()

Testing with a=0, b=0
Testing with a=-2320, b=0
Testing with a=-2320, b=-30740
Testing with a=21, b=0
Testing with a=21, b=7313232499346789569
Testing with a=95, b=0
Testing with a=95, b=-13137
Testing with a=4156, b=0
Testing with a=4156, b=-65
Testing with a=-67, b=0
Testing with a=-67, b=-9
Testing with a=-67, b=-67
Testing with a=-444151752, b=-26114
Testing with a=-26114, b=-26114
Testing with a=10506, b=95
Testing with a=95, b=95
Testing with a=-26054, b=2125298521
Testing with a=2125298521, b=2125298521
Testing with a=-11, b=63
Testing with a=-11, b=-11
Testing with a=11694, b=-29465
Testing with a=11694, b=11694
Testing with a=2498, b=-20261
Testing with a=-20261, b=-20261
Testing with a=-60, b=-9267
Testing with a=-60, b=-60
Testing with a=4, b=-532280228
Testing with a=-532280228, b=-532280228
Testing with a=100, b=-6964
Testing with a=-6964, b=-6964
Testing with a=-111, b=10559
Testing with a=10559, b=10559
Testing with a=-1716849598, b=-92
Testing with a=-1716849598, b=-171

Notice how Hypothesis tries a wide range of integers: positives, negatives, and zero. If the assertion had failed, Hypothesis would have reported it.

## 2. Shrinking, Counter-Examples, and Replaying

Hypothesis's superpower is finding the *simplest possible* failing example. This is called **shrinking**. Let's introduce a bug into a sorting function.

In [2]:
from hypothesis import find, strategies as st, settings

def buggy_sort(numbers):
    # This sort is buggy! It fails if there's a zero.
    for x in numbers:
        if x == 0:
            return [-1] # Return a wrong result
    return sorted(numbers)

# Define a strategy for lists of integers
list_strategy = st.lists(st.integers())

# We use `find` here to demonstrate the counter-example without raising an error
def check_sort(numbers):
    assert buggy_sort(numbers) == sorted(numbers)

failing_example = find(list_strategy, check_sort)
print(f"Hypothesis found a simple failing example: {failing_example}")

AssertionError: 

Hypothesis probably found `[0]`. It might have started with something complex like `[10, 5, 0, -20]`, found that it failed, and then *shrunk* it down to the simplest possible case that still causes the error.

#### Replaying Failures

When a test fails, Hypothesis saves the failing example. You can use the `@example` decorator to add it as a permanent regression test case, ensuring the bug never comes back.

In [3]:
from hypothesis import given, example

@given(st.lists(st.integers()))
@example([0]) # Add the failing case we found!
def test_buggy_sort(numbers):
    # This will now fail immediately with our example if the bug isn't fixed.
    try:
        assert buggy_sort(numbers) == sorted(numbers)
    except AssertionError:
        # In a real test suite, you wouldn't catch the error.
        # We do it here to prevent the notebook from stopping.
        pass

print("Test with @example ran.")

Test with @example ran.


## 3. More Features: Filters, Combining, and `assume()`

You can easily refine strategies.

In [4]:
from hypothesis import assume

# 1. Filtering: Create a strategy for positive integers
positive_integers = st.integers().filter(lambda x: x > 0)

# 2. Combining: Create a strategy for integers OR text
int_or_text = st.one_of(st.integers(), st.text())

# 3. `data()` and `assume()`: For complex, conditional logic inside your test
@given(st.data())
def test_division(data):
    # Use data.draw() to pull from a strategy
    numerator = data.draw(st.integers())
    denominator = data.draw(st.integers())
    
    # `assume` tells Hypothesis to discard this example and try a new one
    assume(denominator != 0)
    
    print(f"Testing {numerator} / {denominator}")
    assert isinstance(numerator / denominator, float)

test_division()

Testing -107 / -21080
Testing 6880 / 20
Testing -46 / 19762
Testing -6032084173671111190 / 5249410788051267593
Testing -23985 / -21796
Testing 16 / 100
Testing 32338 / -61
Testing 1211 / 118
Testing -59 / -31829
Testing -26857 / -5709
Testing -26857 / -26857
Testing 6500 / 1287232088
Testing 1287232088 / 1287232088
Testing 23651 / -30426
Testing 23651 / 23651
Testing 32362 / -5097
Testing 32362 / 32362
Testing -30571 / -4001218974137918807
Testing -4001218974137918807 / -4001218974137918807
Testing 6640226095432587079 / 11624
Testing 11624 / 11624
Testing -7111 / 26674
Testing 26674 / 26674
Testing -24737 / -2956470570003366679
Testing -24737 / -24737
Testing -14149 / -26742
Testing -14149 / -14149
Testing 73 / -1998
Testing -1998 / -1998
Testing 29 / -11445
Testing -11445 / -11445
Testing -20260 / 107
Testing -20260 / -20260
Testing -116 / -28
Testing -28 / -28
Testing 28600 / -534621554
Testing 28600 / 28600
Testing -28396 / 1297009681
Testing 1297009681 / 1297009681
Testing 18994 / 

`assume()` is very powerful. It lets you reject test data that doesn't meet a precondition, without causing the test to fail.

## 4. Custom and Composite Strategies

What if you need to generate custom objects? The `@composite` decorator lets you build new strategies from existing ones.

In [ ]:
from hypothesis.strategies import composite
from dataclasses import dataclass

@dataclass
class User:
    id: int
    name: str

@composite
def user_strategy(draw):
    """A strategy to generate User objects."""
    # `draw` works like `data.draw()` from the previous example
    user_id = draw(st.integers(min_value=1))
    user_name = draw(st.text(min_size=3, max_size=20))
    return User(id=user_id, name=user_name)

@given(user=user_strategy())
def test_user_creation(user):
    print(f"Generated User: {user}")
    assert isinstance(user, User)
    assert user.id > 0
    assert len(user.name) >= 3

test_user_creation()

## 5. Stateful Testing

Hypothesis can even test stateful systems by generating sequences of actions and checking that invariants (rules that should always be true) hold.

Let's test a simple `SimpleStack` class.

In [5]:
from hypothesis.stateful import RuleBasedStateMachine, rule, precondition

class SimpleStack:
    """A basic stack implementation."""
    def __init__(self):
        self._items = []
    
    def push(self, item):
        self._items.append(item)
        
    def pop(self):
        if not self._items:
            raise IndexError("pop from empty list")
        return self._items.pop()
    
    def is_empty(self):
        return not self._items
    
    @property
    def size(self):
        return len(self._items)

class StackStateMachine(RuleBasedStateMachine):
    """Defines the rules for testing our stack."""
    def __init__(self):
        super().__init__()
        self.stack = SimpleStack()
        self.model = [] # A simple list to model the stack's behavior
        
    @rule(item=st.integers())
    def push_item(self, item):
        """Rule for pushing an item."""
        self.stack.push(item)
        self.model.append(item)
        print(f"Pushed {item}")
        
    @rule()
    @precondition(lambda self: not self.stack.is_empty()) # Only run pop if not empty
    def pop_item(self):
        """Rule for popping an item."""
        popped_stack = self.stack.pop()
        popped_model = self.model.pop()
        print(f"Popped {popped_stack}")
        assert popped_stack == popped_model
    
    @rule()
    def check_invariants(self):
        """This rule checks properties that should always be true."""
        print("Checking invariants...")
        assert self.stack.size == len(self.model)
        assert self.stack.is_empty() == (not self.model)

# To run this, Hypothesis would execute a sequence of the rules defined above.
TestStack = StackStateMachine.TestCase
TestStack.runTest = lambda self: None # Suppress unittest's default runner

test_case = TestStack()
test_case.execute_step = test_case.execute_step
print("Running stateful test...")
# Manually run a few steps for demonstration
try:
    for i in range(10): # Run 10 random steps
        test_case.execute_step(test_case.steps.pop(0))
except (IndexError, AttributeError):
    pass # Stop if we run out of generated steps
print("\nStateful test finished.")

AttributeError: 'StackStateMachine.TestCase' object has no attribute 'execute_step'

epic thanks to the pogg0 goats Henry Shu, Inesh Chakrabarti, Ingrid Lee, Nathan Wei, Pritam Mukhopadhyay  